In [ ]:
import h5py
f = h5py.File("/content/drive/My Drive/CTW2020/Processed Data/udata2020.hdf5","r")
import numpy as np
X = np.zeros((31848,56,924),dtype=np.dtype("<f2"))
for i in range(31848):
  temp = f["H_Est"][i:i+1]
  if(i%2000)==0:
    print(i)
  X[i] = temp[:,:,:,0]**2 + temp[:,:,:,1]**2
f.close()

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000


In [ ]:
X.shape

(31848, 56, 924)

In [ ]:
import h5py
f = h5py.File("/content/drive/My Drive/CTW2020/Processed Data/Labelled_1.hdf5","r")
H = f["H_Est"][:]
Pos = f["Pos"][:]
SNR = f["SNR"][:]
f.close()

In [ ]:
import numpy as np
H = np.sqrt(H[:,:,:,0]**2 + H[:,:,:,1]**2)

In [ ]:
from sklearn.model_selection import train_test_split
H_Train, H_Test , Pos_Train , Pos_Test = train_test_split(H,Pos,test_size=0.05, random_state=42)

In [ ]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import os
from torchvision.utils import save_image
from torch.autograd import Variable

In [ ]:
H_Train, H_Val , Pos_Train , Pos_Val = train_test_split(H_Train,Pos_Train,test_size=0.1, random_state=99)

In [ ]:
H_Train = torch.tensor(H_Train,dtype=torch.float)
Pos_Train = torch.tensor(Pos_Train,dtype=torch.float)
H_Val = torch.tensor(H_Val,dtype=torch.float)
Pos_Val = torch.tensor(Pos_Val,dtype=torch.float)

In [ ]:
class Sammon(nn.Module):
    def __init__(self):
        super(Sammon, self).__init__()
        
        
        self.main = nn.Sequential(
            nn.Conv2d(1, 3, (4,2), stride=(4,1)),
            nn.ReLU(),
            nn.Conv2d(3, 6, (4,2), stride=(4,1)),
            nn.ReLU())
        
        self.Project = nn.Sequential(nn.Linear(16596,7000),
                                     nn.Tanh(),
                                     nn.Linear(7000,5000),
                                     nn.Softplus(),
                                     nn.Linear(5000,4500),
                                     nn.Tanh(),)
                                     
          
            
    def forward(self, input):
        return self.Project(self.main(input).view(-1,16596))


class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()        
        self.Project = nn.Sequential(nn.Linear(4500,2048),
                                     nn.Tanh(),
                                     nn.Linear(2048,1024),
                                     nn.Softplus(),
                                     nn.Linear(1024,512),
                                     nn.Tanh(),
                                     nn.Linear(512,256),
                                     nn.Tanh(),
                                     nn.Linear(256,128),
                                     nn.Tanh(),
                                     nn.Linear(128,64),
                                     nn.Tanh(),
                                     nn.Linear(64,32),
                                     nn.Softplus(),
                                     nn.Linear(32,16),
                                     nn.Tanh(),
                                     nn.Linear(16,8),
                                     nn.Tanh(),
                                     nn.Linear(8,3))
          
            
    def forward(self, input):
        return self.Project(input).view(-1,3)


In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
sammon = Sammon().to(device)
reg = Regressor().to(device)

In [ ]:
opt1 = optim.Adam(sammon.parameters(), lr=1e-4, betas=(0.5, 0.999))
opt2 = optim.Adam(reg.parameters(), lr=1e-4, betas=(0.5, 0.999))
MSE = loss = nn.MSELoss()

In [ ]:
A = list(range(len(X)))*4
pairs = []
import random
while len(A)>1:
  t = random.randint(1,len(A)-1)
  if(A[t]!=A[0]):
    pairs.append((A[0],A[t]))
    del A[0]
    del A[t-1]


In [ ]:
pairs = np.array(pairs)

In [ ]:
N_b2 = len(H_Train)
N_b = len(pairs)
N_val = len(H_Val)
bs1= 64
bs2 = 8

In [ ]:
for i in range(100):
  batch1=0
  batch2=0
  for j in range(0,N_b,bs1):
    sammon.zero_grad()
    end = min(j+bs1,N_b)
    Xn,Xm = torch.tensor(X[pairs[j:end,0]],dtype=torch.float),  torch.tensor(X[pairs[j:end,1]],dtype=torch.float)
    Xn , Xm = Variable(Xn, requires_grad=True) , Variable(Xm, requires_grad=True)
    Xn = Xn.to(device)
    Xm = Xm.to(device)
    Yn = sammon(Xn.view(-1,1,56,924)).view(-1)
    Ym = sammon(Xm.view(-1,1,56,924)).view(-1)
    loss1 = MSE(Xn, Xm)
    loss2 = MSE(Yn,Ym)
    loss = MSE(loss1,loss2)
    loss.backward()
    opt1.step()
    batch1+=1
    if(batch1%10 == 0):
      print("Batch",batch1,"/",(N_b//bs1)+1,"Sammon Loss :" , loss.item())
  
  for k in range(0,N_b2,bs2):
    end = min(k+bs2,N_b2)
    H1 = H_Train[k:end]
    sammon.zero_grad()
    reg.zero_grad()
    H1 = Variable(H1,requires_grad=True)
    H1 = H1.to(device)
    Y1 = reg(sammon(H1.view(-1,1,56,924)))
    loss_1 = MSE(Y1,Pos_Train[k:end].to(device)).view(-1)
    loss_1.backward()
    opt1.step()
    opt2.step()
    batch2+=1
    if(batch2%10 == 0):
      print("Batch",batch2,"/",(N_b2//bs2)+1,"Regressor Loss :" , loss_1.item())
  print("Epoch ",i,"/",100," Sammon Loss :",loss.item() ," Regressor Loss :" , loss_1.item())
  for l in range(N_val):
    H_val = torch.tensor(H_Val[l],dtype=torch.float)
    H_val = H_val.to(device)
    with torch.no_grad():
      Y_Val = reg(sammon(H_val.view(-1,1,56,924)))
      loss_Val = MSE(Y_Val,Pos_Val[l].to(device)).view(-1)
  print("Val Loss :" , loss_Val.item())

    

      
  


Batch 10 / 996 Sammon Loss : 0.0083991177380085
Batch 20 / 996 Sammon Loss : 0.0010822005569934845
Batch 30 / 996 Sammon Loss : 0.006653514225035906
Batch 40 / 996 Sammon Loss : 0.032325953245162964
Batch 50 / 996 Sammon Loss : 0.021435728296637535
Batch 60 / 996 Sammon Loss : 0.02428772859275341
Batch 70 / 996 Sammon Loss : 0.010107041336596012
Batch 80 / 996 Sammon Loss : 0.016238121315836906
Batch 90 / 996 Sammon Loss : 0.001901067211292684
Batch 100 / 996 Sammon Loss : 0.012429174967110157
Batch 110 / 996 Sammon Loss : 0.00011216865823371336
Batch 120 / 996 Sammon Loss : 0.009747683070600033
Batch 130 / 996 Sammon Loss : 0.0007925923564471304
Batch 140 / 996 Sammon Loss : 0.004083746112883091
Batch 150 / 996 Sammon Loss : 0.00011599066783674061
Batch 160 / 996 Sammon Loss : 0.0017142882570624352
Batch 170 / 996 Sammon Loss : 0.005111721809953451
Batch 180 / 996 Sammon Loss : 0.028658824041485786
Batch 190 / 996 Sammon Loss : 0.03912125900387764
Batch 200 / 996 Sammon Loss : 0.00230

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Val Loss : 108700.53125
Batch 10 / 996 Sammon Loss : 0.20362131297588348
Batch 20 / 996 Sammon Loss : 0.2075655460357666
Batch 30 / 996 Sammon Loss : 0.3207903206348419
Batch 40 / 996 Sammon Loss : 0.943100094795227
Batch 50 / 996 Sammon Loss : 0.6516808867454529
Batch 60 / 996 Sammon Loss : 0.05872718617320061
Batch 70 / 996 Sammon Loss : 0.06931544840335846
Batch 80 / 996 Sammon Loss : 0.050553422421216965
Batch 90 / 996 Sammon Loss : 0.020151996985077858
Batch 100 / 996 Sammon Loss : 0.016281787306070328
Batch 110 / 996 Sammon Loss : 0.008399019949138165
Batch 120 / 996 Sammon Loss : 0.046326540410518646
Batch 130 / 996 Sammon Loss : 0.001967438729479909
Batch 140 / 996 Sammon Loss : 0.02190415933728218
Batch 150 / 996 Sammon Loss : 0.00015732993779238313
Batch 160 / 996 Sammon Loss : 0.006906980648636818
Batch 170 / 996 Sammon Loss : 0.21873237192630768
Batch 180 / 996 Sammon Loss : 0.07931903749704361
Batch 190 / 996 Sammon Loss : 0.21199102699756622
Batch 200 / 996 Sammon Loss : 

KeyboardInterrupt: ignored

In [ ]:
pairs[2:9,0]

array([2, 3, 4, 5, 6, 7, 8])